In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

file_path_2 = '/Users/yasudayuuya/orderpredict/yosoku_yasuda/【2月】Bライン生産性結果記入用紙_Ver.4.3.xlsx'  # Excelファイルのパスを指定
sheet_name = '実績昼'  # 読み込むシート名を指定
data_2 = pd.read_excel(file_path_2, sheet_name=sheet_name)

file_path_3 = '/Users/yasudayuuya/orderpredict/yosoku_yasuda/【3月】Bライン生産性結果記入用紙_Ver.4.3.xlsx'  # Excelファイルのパスを指定
sheet_name = '実績昼'  # 読み込むシート名を指定
data_3 = pd.read_excel(file_path_3, sheet_name=sheet_name)


/Users/yasudayuuya/orderpredict/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 行列を逆にする
data_2 = data_2.T
# 1行目を列名にする
data_2.columns = data_2.iloc[1]
# 0,1行目を削除
data_2 = data_2.drop(data_2.index[[0, 1]])

# 行列を逆にする
data_3 = data_3.T
# 1行目を列名にする
data_3.columns = data_3.iloc[1]
# 0,1行目を削除
data_3 = data_3.drop(data_3.index[[0, 1]])

In [3]:
# 欠損値を探す
data_2.isnull().sum()
data_3.isnull().sum()

Unnamed: 1
NaN            32
NaN            32
項目              0
件数              0
作業時間(分)         0
作業人数           20
リーダー人数         20
合計作業人数          3
作業ライン          20
1人あたりの作業時間     21
1本あたりの平均作業数    21
1日あたりの作業件数     31
日付              1
dtype: int64

In [7]:
data_3.head()


Unnamed: 1,NaN,NaN,項目,件数,作業時間(分),作業人数,リーダー人数,合計作業人数,作業ライン,1人あたりの作業時間,1本あたりの平均作業数,1日あたりの作業件数,日付
Unnamed: 2,NaN,NaN,2024-10-01 00:00:00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
Unnamed: 3,NaN,NaN,2024-10-02 00:00:00,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,2
Unnamed: 4,NaN,NaN,2024-10-03 00:00:00,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,3
Unnamed: 5,NaN,NaN,2024-10-04 00:00:00,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,4
Unnamed: 6,NaN,NaN,2024-10-05 00:00:00,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,5


In [8]:
data_2 = data_2.drop(data_2.columns[[0, 1, 9, 10]], axis=1)
data_3 = data_3.drop(data_3.columns[[0, 1, 2, 10, 11]], axis=1)

In [9]:
# NaNを全て0に変換
data_2 = data_2.fillna(0)
data_3 = data_3.fillna(0)

/var/folders/wl/_sk4lq9107v79smv_pqvls780000gn/T/ipykernel_4947/2367021029.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_2 = data_2.fillna(0)
/var/folders/wl/_sk4lq9107v79smv_pqvls780000gn/T/ipykernel_4947/2367021029.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_3 = data_3.fillna(0)


In [12]:
# 0列目のデータを表示
data_2.iloc[:, 0]

Unnamed: 2        0
Unnamed: 3        0
Unnamed: 4        0
Unnamed: 5        0
Unnamed: 6        0
Unnamed: 7      177
Unnamed: 8      294
Unnamed: 9      177
Unnamed: 10       0
Unnamed: 11     380
Unnamed: 12     640
Unnamed: 13     481
Unnamed: 14     530
Unnamed: 15     569
Unnamed: 16       0
Unnamed: 17       0
Unnamed: 18     597
Unnamed: 19     417
Unnamed: 20     202
Unnamed: 21       0
Unnamed: 22       0
Unnamed: 23       0
Unnamed: 24       0
Unnamed: 25       0
Unnamed: 26       0
Unnamed: 27       0
Unnamed: 28       0
Unnamed: 29       0
Unnamed: 30       0
Unnamed: 31       0
Unnamed: 32       0
Unnamed: 33    4464
Name: 件数, dtype: int64

In [13]:
data_3.iloc[:, 0]

Unnamed: 2        0
Unnamed: 3        0
Unnamed: 4        0
Unnamed: 5        0
Unnamed: 6        0
Unnamed: 7        0
Unnamed: 8      237
Unnamed: 9        0
Unnamed: 10       0
Unnamed: 11     423
Unnamed: 12     400
Unnamed: 13     311
Unnamed: 14     440
Unnamed: 15     493
Unnamed: 16      90
Unnamed: 17       0
Unnamed: 18     801
Unnamed: 19     281
Unnamed: 20     502
Unnamed: 21     231
Unnamed: 22       0
Unnamed: 23       0
Unnamed: 24       0
Unnamed: 25       0
Unnamed: 26       0
Unnamed: 27       0
Unnamed: 28       0
Unnamed: 29       0
Unnamed: 30       0
Unnamed: 31       0
Unnamed: 32       0
Unnamed: 33    4209
Name: 件数, dtype: int64

In [14]:
# Unnamed: 33と34を削除
data_2 = data_2.drop(data_2.index[[28, 29, 30, 31]])
data_3 = data_3.drop(data_3.index[[31]])


In [15]:
# 「件数」数が0の行を削除
data_2 = data_2[data_2['件数'] != 0]
data_3 = data_3[data_3['件数'] != 0]

In [16]:
# データの標準偏差及び平均を取得（正規化解除に使用）
original_mean_2 = data_2.mean()
original_std_2 = data_2.std()
print(original_mean_2)
print(original_std_2)

Unnamed: 1
件数             405.818182
作業時間(分)       2980.272727
作業人数             7.545455
リーダー人数           2.181818
合計作業人数           9.727273
作業ライン            3.000000
1人あたりの作業時間     300.119697
日付              12.272727
dtype: float64
Unnamed: 1
件数             172.813667
作業時間(分)       1260.490150
作業人数             1.863525
リーダー人数           0.603023
合計作業人数           2.284334
作業ライン            0.774597
1人あたりの作業時間     101.800955
日付               4.429242
dtype: float64


In [17]:
# データの標準偏差及び平均を取得（正規化解除に使用）
original_mean_3 = data_3.mean()
original_std_3 = data_3.std()
print(original_mean_3)
print(original_std_3)

Unnamed: 1
件数             382.636364
作業時間(分)       2788.181818
作業人数             1.545455
リーダー人数           7.000000
合計作業人数           8.545455
作業ライン            2.818182
1人あたりの作業時間     321.000000
日付              14.181818
dtype: float64
Unnamed: 1
件数             187.610913
作業時間(分)       1295.128628
作業人数             0.522233
リーダー人数           2.569047
合計作業人数           2.978712
作業ライン            0.981650
1人あたりの作業時間      95.838210
日付               4.069845
dtype: float64


In [18]:
# 対数変換
data_2 = data_2.apply(lambda x: np.log(x + 1))
data_3 = data_3.apply(lambda x: np.log(x + 1))

In [19]:
# カラム名の順番を合わせる
data_2 = data_2[data_3.columns]

In [20]:
# 目的変数と説明変数の設定
X_train = data_2.drop(columns='件数', errors='ignore')
y_train = data_2['件数']
X_test = data_3.drop(columns='件数', errors='ignore')
y_test = data_3['件数']

# ベイズ最適化

In [27]:
%pip install bayesian-optimization
from bayes_opt import BayesianOptimization
import joblib
from sklearn.model_selection import cross_val_score

def optimize_gbr_cv(n_estimators, learning_rate, max_depth, max_features, min_samples_split, subsample):
    model = GradientBoostingRegressor(
        n_estimators=int(n_estimators),
        learning_rate=learning_rate,
        max_depth=int(max_depth),
        max_features=max_features,
        min_samples_split=int(min_samples_split),
        subsample=subsample,
        random_state=42
    )
    # CVでの平均R²スコアを返す
    return cross_val_score(model, X_train, y_train, cv=5, scoring='r2').mean()

# ハイパーパラメータの範囲を設定
pbounds = {
    'n_estimators': (100, 1000),
    'learning_rate': (0.005, 0.3),
    'max_depth': (3, 10),
    'min_samples_split': (2, 20),
    'subsample': (0.5, 1.0),
    'max_features': (0.5, 1.0)

}

# ベイズ最適化の設定および実行
optimizer = BayesianOptimization(
    f=optimize_gbr_cv,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

optimizer.maximize(init_points=5, n_iter=25)

# 最良のパラメータを取得し、最適なモデルを作成
best_params = optimizer.max['params']
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['min_samples_split'] = int(best_params['min_samples_split'])
best_params['subsample'] = best_params['subsample']
best_params['max_features'] = best_params['max_features']


print("Best parameters found: ", best_params)

best_model = GradientBoostingRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# テストデータでの予測
y_pred = best_model.predict(X_test)

# 精度の評価
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Test Mean Squared Error: {mse}')
print(f'Test R² Score: {r2}')


# モデル保存（拡張子は .pkl が多い）
# 学習に使った特徴量（カラム名と順番）を保存しておく
feature_names = X_train.columns.tolist()
joblib.dump((best_model, feature_names), 'model.pkl2')

print("モデルを保存しました。")

  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
|   iter    |  target   | learni... | max_depth | max_fe... | min_sa... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | -113.2    | 0.1155    | 9.655     | 0.866     | 12.78     | 240.4     | 0.578     |
| 2         | -124.7    | 0.02213   | 9.063     | 0.8006    | 14.75     | 118.5     | 0.985     |
| 3         | -72.08    | 0.2506    | 4.486     | 0.5909    | 5.301     | 373.8     | 0.7624    |
| 4         | -61.51    | 0.1324    | 5.039     | 0.8059    | 4.511     | 362.9     | 0.6832    |
| 5         | -124.4    | 0.1395    | 8.496     | 0.5998    | 11.26     | 633.2     | 0.5232    |
| 6       

In [28]:
# 全てのデータに対して
# 予測を行う
y_pred = best_model.predict(X_test)

# 予測結果を元に戻す
y_pred_original = np.exp(y_pred)
y_test_original = np.exp(y_test)

# 各インデックスごとに比較結果を表示
for i in range(len(X_test)):
    print(f'Predicted: {y_pred_original[i]}')
    print(f'Actual: {y_test_original.iloc[i]}')
    print(f'Difference: {y_pred_original[i] - y_test_original.iloc[i]}')
    print('----------')

Predicted: 172.6438068392298
Actual: 238.00000000000009
Difference: -65.35619316077029
----------
Predicted: 244.25523885391453
Actual: 424.0
Difference: -179.74476114608547
----------
Predicted: 365.3528864884705
Actual: 401.00000000000006
Difference: -35.64711351152954
----------
Predicted: 233.53371964983927
Actual: 312.00000000000006
Difference: -78.46628035016079
----------
Predicted: 413.56387965214117
Actual: 441.0
Difference: -27.436120347858832
----------
Predicted: 417.39008172334593
Actual: 494.00000000000017
Difference: -76.60991827665424
----------
Predicted: 185.9223245521382
Actual: 90.99999999999997
Difference: 94.92232455213824
----------
Predicted: 394.5704698862117
Actual: 802.0000000000002
Difference: -407.42953011378853
----------
Predicted: 398.65867242262254
Actual: 282.00000000000006
Difference: 116.65867242262249
----------
Predicted: 236.0151938907365
Actual: 503.0
Difference: -266.9848061092635
----------
Predicted: 232.93962685653062
Actual: 231.999999999999

In [29]:
# 表にして表示
df = pd.DataFrame({'Predicted': y_pred_original, 'Actual': y_test_original, 'Difference': y_pred_original - y_test_original})
df

,Predicted,Actual,Difference
Unnamed: 8,172.643807,238.0,-65.356193
Unnamed: 11,244.255239,424.0,-179.744761
Unnamed: 12,365.352886,401.0,-35.647114
Unnamed: 13,233.533720,312.0,-78.466280
Unnamed: 14,413.563880,441.0,-27.436120
Unnamed: 15,417.390082,494.0,-76.609918
Unnamed: 16,185.922325,91.0,94.922325
Unnamed: 18,394.570470,802.0,-407.429530
Unnamed: 19,398.658672,282.0,116.658672
Unnamed: 20,236.015194,503.0,-266.984806


In [30]:
# differenceの絶対値を合計
df['Difference'].abs().sum()
print(df['Difference'].abs().sum())

1350.1953468474026


In [31]:
# differenceの合計
df['Difference'].sum()
print(df['Difference'].sum())

-925.1540991848199


In [32]:
# differenceの標準偏差を取得
df['Difference'].std()
print(df['Difference'].std())

152.5513620644319


In [33]:
# differenceの平均
df['Difference'].mean()
print(df['Difference'].mean())


-84.1049181077109


In [34]:
# 予測結果がプラス・マイナス100件に収まっている数を数える
# プラス・マイナス100件に収まっている数をカウントする変数
within_100_count = 0

# 各インデックスごとに比較結果を表示
for i in range(len(X_test)):
    diff = y_pred_original[i] - y_test_original.iloc[i]
    
    # 差がプラス・マイナス100件に収まっている場合
    if abs(diff) <= 100:
        within_100_count += 1

# 結果を表示
print(f'プラス・マイナス100件に収まっている数: {within_100_count}')

プラス・マイナス100件に収まっている数: 7


In [35]:
# 予測結果がプラス・マイナス100件に収まっている数を数える
# プラス・マイナス100件に収まっている数をカウントする変数
within_200_count = 0

# 各インデックスごとに比較結果を表示
for i in range(len(X_test)):
    diff = y_pred_original[i] - y_test_original.iloc[i]
    
    # 差がプラス・マイナス200件に収まっている場合
    if abs(diff) <= 200:
        within_200_count += 1

# 結果を表示
print(f'プラス・マイナス200件に収まっている数: {within_200_count}')

プラス・マイナス200件に収まっている数: 9


In [36]:
# 特徴量重要度を表示
feature_importances = best_model.feature_importances_
features = data_2.drop(columns='件数', errors='ignore').columns
df = pd.DataFrame({'features': features, 'importance': feature_importances})
df = df.sort_values(by='importance', ascending=False)
df

,features,importance
1,作業人数,0.438873
4,作業ライン,0.250497
6,日付,0.137720
0,作業時間(分),0.104909
5,1人あたりの作業時間,0.049668
3,合計作業人数,0.016492
2,リーダー人数,0.001840
